# Pre-ETL de OpenFoodFacts (OFF)

En este notebook se realiza el **preprocesado pesado** del archivo original de OpenFoodFacts:

- Archivo de entrada: `data_raw/openfoodfacts/en.openfoodfacts.org.products.csv`  
- Tamaño aproximado: ~11 GB  
- **Objetivo:** generar un subconjunto reducido con:
  - Sólo productos asociados a países de la **Unión Europea (UE-27)**,
  - Variables nutricionales clave (energía, azúcares, grasas saturadas y sodio),
  - Periodo aproximado **2015–2023** según la fecha de modificación (`last_modified_t`),
  - Guardado en formato **Parquet** para facilitar los análisis posteriores.

Este pre-ETL se ejecuta una sola vez y el resultado se reutiliza en los notebooks exploratorios y en el dashboard interactivo.

In [2]:
from pathlib import Path
import duckdb

# Carpeta raíz del proyecto
ROOT_DIR = Path("..").resolve().parent

# Carpetas de datos crudos y procesados
DATA_RAW = ROOT_DIR / "data_raw" / "openfoodfacts"
DATA_PROCESSED = ROOT_DIR / "data_processed" / "openfoodfacts"

# Archivo CSV original de OpenFoodFacts
CSV_PATH = DATA_RAW / "en.openfoodfacts.org.products.csv"

ROOT_DIR, DATA_RAW, DATA_PROCESSED, CSV_PATH

(WindowsPath('C:/Users/santi/OneDrive - UNIR/UNIR/MASTER ANÁLISIS Y VISUALIZACIÓN BIG DATA/TFM/dashboard-coherencia-ue-tfm'),
 WindowsPath('C:/Users/santi/OneDrive - UNIR/UNIR/MASTER ANÁLISIS Y VISUALIZACIÓN BIG DATA/TFM/dashboard-coherencia-ue-tfm/data_raw/openfoodfacts'),
 WindowsPath('C:/Users/santi/OneDrive - UNIR/UNIR/MASTER ANÁLISIS Y VISUALIZACIÓN BIG DATA/TFM/dashboard-coherencia-ue-tfm/data_processed/openfoodfacts'),
 WindowsPath('C:/Users/santi/OneDrive - UNIR/UNIR/MASTER ANÁLISIS Y VISUALIZACIÓN BIG DATA/TFM/dashboard-coherencia-ue-tfm/data_raw/openfoodfacts/en.openfoodfacts.org.products.csv'))

In [3]:
CSV_PATH.exists(), CSV_PATH

(True,
 WindowsPath('C:/Users/santi/OneDrive - UNIR/UNIR/MASTER ANÁLISIS Y VISUALIZACIÓN BIG DATA/TFM/dashboard-coherencia-ue-tfm/data_raw/openfoodfacts/en.openfoodfacts.org.products.csv'))

## 1. Comprobación de lectura del CSV con DuckDB

Antes de hacer transformaciones, comprobamos que con DuckDB se puede:

- Leer el archivo CSV con el separador correcto (`\t` = tabulador).
- Interpretar automáticamente los tipos de datos.
- Mostrar unas pocas filas de ejemplo.

Con esto no cargamos todo el archivo en memoria, sino que sólo hacemos una lectura parcial.

In [4]:
# Creamos conexión en memoria
con = duckdb.connect(database=":memory:")

preview_df = con.execute(f"""
    SELECT *
    FROM read_csv_auto('{CSV_PATH}', delim='\t', header=TRUE)
    LIMIT 5
""").fetchdf()

preview_df

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,...,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g,carbohydrates-total_100g
0,00000002,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1760861583,2025-10-19 03:13:03-05:00,1760861586,2025-10-19 03:13:06-05:00,None,1760861586,2025-10-19 03:13:06-05:00,...,None,<NA>,None,None,None,None,None,None,None,None
1,00000003,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1752485388,2025-07-14 04:29:48-05:00,1752485389,2025-07-14 04:29:49-05:00,None,1752485389,2025-07-14 04:29:49-05:00,...,None,<NA>,None,None,None,None,None,None,None,None
2,00000004,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1763548527,2025-11-19 05:35:27-05:00,1763549702,2025-11-19 05:55:02-05:00,smoothie-app,1763549702,2025-11-19 05:55:02-05:00,...,None,<NA>,None,None,None,None,None,None,None,None
3,00000005,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1754314021,2025-08-04 08:27:01-05:00,1754314023,2025-08-04 08:27:03-05:00,None,1754314023,2025-08-04 08:27:03-05:00,...,None,<NA>,None,None,None,None,None,None,None,None
4,00000006,http://world-en.openfoodfacts.org/product/0000...,moon-rabbit,1760212975,2025-10-11 15:02:55-05:00,1760218930,2025-10-11 16:42:10-05:00,ascharao,1760218930,2025-10-11 16:42:10-05:00,...,None,<NA>,None,None,None,None,None,None,None,None


## 2. Inspección del esquema y selección de variables clave

El archivo de OpenFoodFacts tiene más de 200 columnas.  
En este pre-ETL solo necesitamos quedarnos con:

- Identificadores básicos del producto (código, nombre, marcas),
- Información de países y categorías,
- Nutrientes necesarios para el índice nutricional,
- Algunas variables de apoyo (fechas, clasificación NOVA, Nutri-Score, etc.).

Usamos DuckDB para consultar el esquema completo y luego definimos
una lista de columnas a conservar.

In [5]:
# Esquema de columnas y tipos inferidos por DuckDB
schema_df = con.execute(f"""
    DESCRIBE
    SELECT *
    FROM read_csv_auto('{CSV_PATH}', delim='\t', header=TRUE)
""").fetchdf()

# Mostramos las primeras 40 filas
schema_df.head(40)

,column_name,column_type,null,key,default,extra
0,code,VARCHAR,YES,None,None,None
1,url,VARCHAR,YES,None,None,None
2,creator,VARCHAR,YES,None,None,None
3,created_t,BIGINT,YES,None,None,None
4,created_datetime,TIMESTAMP WITH TIME ZONE,YES,None,None,None
5,last_modified_t,BIGINT,YES,None,None,None
6,last_modified_datetime,TIMESTAMP WITH TIME ZONE,YES,None,None,None
7,last_modified_by,VARCHAR,YES,None,None,None
8,last_updated_t,BIGINT,YES,None,None,None
9,last_updated_datetime,TIMESTAMP WITH TIME ZONE,YES,None,None,None


## 3. Selección de columnas relevantes

Del archivo completo de OpenFoodFacts sólo conservaré:

- Identificación del producto (código, nombre, marcas).
- Países y categorías/grupos de alimentos.
- Nutrientes necesarios para el índice nutricional.
- Indicadores de calidad nutricional y procesado.
- Fechas básicas de creación/modificación.

Con ello se puede generar un recorte en formato Parquet para trabajar de forma
más ligera en los análisis y en el dashboard.

In [6]:
# Lista de columnas que queremos conservar del CSV original
cols_keep = [
    # Identificación básica
    "code",
    "product_name",
    "brands",

    # Países y grupos
    "countries",
    "countries_tags",
    "categories",
    "categories_tags",
    "pnns_groups_1",
    "pnns_groups_2",

    # Cantidades
    "quantity",
    "serving_size",

    # Nutrientes clave para el índice
    "energy_100g",
    "sugars_100g",
    "saturated-fat_100g",
    "sodium_100g",
    "salt_100g",
    "fat_100g",
    "carbohydrates_100g",
    "fiber_100g",
    "proteins_100g",

    # Calidad nutricional / procesado
    "nova_group",

    # Trazabilidad básica
    "created_datetime",
    "last_modified_datetime",
]

len(cols_keep)

23

In [7]:
# Comprobamos qué columnas de la lista no existen en el CSV
existing_cols = set(schema_df["column_name"])
missing_cols = [c for c in cols_keep if c not in existing_cols]
missing_cols

[]

In [8]:
# Construimos la parte SQL "col1, col2, col3, ..." asegurando comillas
cols_sql = ",\n            ".join(f'"{c}"' for c in cols_keep)
print(cols_sql)

"code",
            "product_name",
            "brands",
            "countries",
            "countries_tags",
            "categories",
            "categories_tags",
            "pnns_groups_1",
            "pnns_groups_2",
            "quantity",
            "serving_size",
            "energy_100g",
            "sugars_100g",
            "saturated-fat_100g",
            "sodium_100g",
            "salt_100g",
            "fat_100g",
            "carbohydrates_100g",
            "fiber_100g",
            "proteins_100g",
            "nova_group",
            "created_datetime",
            "last_modified_datetime"


In [10]:
# Ruta de salida para el archivo Parquet reducido
PARQUET_PATH = DATA_PROCESSED / "openfoodfacts_subset.parquet"
PARQUET_PATH

WindowsPath('C:/Users/santi/OneDrive - UNIR/UNIR/MASTER ANÁLISIS Y VISUALIZACIÓN BIG DATA/TFM/dashboard-coherencia-ue-tfm/data_processed/openfoodfacts/openfoodfacts_subset.parquet')

In [11]:
# Creamos la carpeta de salida
PARQUET_PATH.parent.mkdir(parents=True, exist_ok=True)

In [12]:
# Leemos el CSV gigante y guardamos solo las columnas seleccionadas en Parquet
con.execute(f"""
    COPY (
        SELECT
            {cols_sql}
        FROM read_csv_auto('{CSV_PATH}', delim='\t', header=TRUE)
    )
    TO '{PARQUET_PATH}'
    (FORMAT PARQUET);
""")

In [13]:
PARQUET_PATH.exists(), PARQUET_PATH

(True,
 WindowsPath('C:/Users/santi/OneDrive - UNIR/UNIR/MASTER ANÁLISIS Y VISUALIZACIÓN BIG DATA/TFM/dashboard-coherencia-ue-tfm/data_processed/openfoodfacts/openfoodfacts_subset.parquet'))

In [14]:
off_sample = con.execute(f"""
    SELECT *
    FROM read_parquet('{PARQUET_PATH}')
    LIMIT 5
""").fetchdf()

off_sample

,code,product_name,brands,countries,countries_tags,categories,categories_tags,pnns_groups_1,pnns_groups_2,quantity,...,saturated-fat_100g,sodium_100g,salt_100g,fat_100g,carbohydrates_100g,fiber_100g,proteins_100g,nova_group,created_datetime,last_modified_datetime
0,00000002,None,None,en:Germany,en:germany,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2025-10-19 03:13:03-05:00,2025-10-19 03:13:06-05:00
1,00000003,None,None,en:France,en:france,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2025-07-14 04:29:48-05:00,2025-07-14 04:29:49-05:00
2,00000004,None,None,en:france,en:france,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2025-11-19 05:35:27-05:00,2025-11-19 05:55:02-05:00
3,00000005,None,None,en:France,en:france,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2025-08-04 08:27:01-05:00,2025-08-04 08:27:03-05:00
4,00000006,None,None,en:Germany,en:germany,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,2025-10-11 15:02:55-05:00,2025-10-11 16:42:10-05:00


In [15]:
# Número total de filas en el Parquet reducido
n_rows = con.execute(f"""
    SELECT COUNT(*) AS n_filas
    FROM read_parquet('{PARQUET_PATH}')
""").fetchdf()

n_rows

,n_filas
0,4210261


In [16]:
# Tomamos una muestra de hasta 100.000 filas
sample_df = con.execute(f"""
    SELECT *
    FROM read_parquet('{PARQUET_PATH}')
    LIMIT 100000
""").fetchdf()

# Porcentaje de valores nulos por columna
na_ratio = sample_df.isna().mean().sort_values(ascending=False)

na_ratio

quantity                  0.72654
nova_group                0.49022
categories_tags           0.45430
categories                0.45430
fiber_100g                0.39977
brands                    0.39491
serving_size              0.37332
saturated-fat_100g        0.29285
salt_100g                 0.28120
sodium_100g               0.28120
sugars_100g               0.24537
fat_100g                  0.20540
proteins_100g             0.20478
carbohydrates_100g        0.20228
energy_100g               0.19801
product_name              0.05431
pnns_groups_2             0.00679
pnns_groups_1             0.00679
countries_tags            0.00520
countries                 0.00519
code                      0.00000
created_datetime          0.00000
last_modified_datetime    0.00000
dtype: float64

In [17]:
# 1. DTypes de las 23 columnas en el Parquet
dtypes_df = con.execute(f"""
    SELECT *
    FROM read_parquet('{PARQUET_PATH}')
    LIMIT 0
""").fetchdf()
dtypes_df.dtypes

code                                              object
product_name                                      object
brands                                            object
countries                                         object
countries_tags                                    object
categories                                        object
categories_tags                                   object
pnns_groups_1                                     object
pnns_groups_2                                     object
quantity                                          object
serving_size                                      object
energy_100g                                      float64
sugars_100g                                      float64
saturated-fat_100g                               float64
sodium_100g                                      float64
salt_100g                                        float64
fat_100g                                         float64
carbohydrates_100g             

In [18]:
# 2. Un ejemplo de producto "bien rellenado" de nutrientes
good_example = con.execute(f"""
    SELECT 
        code,
        product_name,
        countries,
        pnns_groups_1,
        pnns_groups_2,
        energy_100g,
        fat_100g,
        "saturated-fat_100g",
        sugars_100g,
        sodium_100g,
        proteins_100g
    FROM read_parquet('{PARQUET_PATH}')
    WHERE energy_100g IS NOT NULL
      AND fat_100g IS NOT NULL
      AND "saturated-fat_100g" IS NOT NULL
      AND sugars_100g IS NOT NULL
      AND sodium_100g IS NOT NULL
    LIMIT 5
""").fetchdf()

good_example

,code,product_name,countries,pnns_groups_1,pnns_groups_2,energy_100g,fat_100g,saturated-fat_100g,sugars_100g,sodium_100g,proteins_100g
0,00000007,granola Bio le Chocolaté,"Spanien, Germany",Fruits and vegetables,Dried fruits,4.0,1.0,1.00,1.00,0.400,1.0
1,00000008,None,"Vereinigte Staaten von Amerika, Germany",unknown,unknown,1510.0,2.0,0.50,1.70,0.600,76.0
2,00000009,xytitol pastilles,"Germany,Spain",Composite foods,Sandwiches,293.0,0.5,0.06,0.24,0.110,18.0
3,00000013,Powdered peanut butter,en:Switzerland,Composite foods,One-dish meals,188.0,13.0,6.70,3.60,0.025,11.0
4,00000015,Madeleines ChocoLait,France,Sugary snacks,Biscuits and cakes,1926.0,24.0,6.00,31.00,0.192,6.4


In [19]:
# 3. Guardar en el notebook un pequeño resumen para el TFM
summary = {
    "n_filas_parquet": int(n_rows["n_filas"][0]),
    "porc_nulos_energia": float(na_ratio["energy_100g"]),
    "porc_nulos_saturadas": float(na_ratio["saturated-fat_100g"]),
    "porc_nulos_azucares": float(na_ratio["sugars_100g"]),
}
summary

{'n_filas_parquet': 4210261,
 'porc_nulos_energia': 0.19801,
 'porc_nulos_saturadas': 0.29285,
 'porc_nulos_azucares': 0.24537}